<a href="https://colab.research.google.com/github/auzaluis/2024-1_UTEPSA_BigData_AnalisisDeDatos/blob/main/Python/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 1: Carga de datos

## **Importando base de datos**

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API Client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread Authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = "https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing"
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet("Respuestas de formulario 1").get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo en un df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns = sheets[0])

## Inspección data frame

In [ ]:
type(df)

In [ ]:
# Visualizando las primeras filas
df.head()

In [ ]:
# Mirada general
df.info()

In [ ]:
# Tipo de datos
df["Sexo"].dtype

In [ ]:
# Número de filas y columnas
df.shape

In [ ]:
# Número de filas
len(df)

In [ ]:
df.columns

In [ ]:
len(df.columns)

# **Tema 2: Transformación de datos**

## **Valores perdidos**

### **Identificación de los NAs**

In [ ]:
# Los NAs en realidad son strings vacíos
import numpy as np
df.replace("", np.nan, inplace = True)

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

### **Omitiendo**



In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors = 'coerce')

In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'].mean()

In [ ]:
edad_promedio = df['Escribe tu edad exacta'].mean()

### **Reemplazo por la media**

In [ ]:
# creando df2
df2 = df.copy()

In [ ]:
# reemplazo por la media
df2["edad2"] = df2["Escribe tu edad exacta"].fillna(edad_promedio)

**Reubicando la columna edad2**

In [ ]:
# Colocar los nombres de las columnas en una lista
lista_columnas = df2.columns.to_list()

In [ ]:
# Quitar temporalmente edad2 de la lista
lista_columnas.remove("edad2")

In [ ]:
# obteniendo el índice de la columna 'after'
lista_columnas.index("Escribe tu edad exacta")

In [ ]:
# insertando 'edad2' después de 'Escribe tu edad exacta'
lista_columnas.insert(lista_columnas.index("Escribe tu edad exacta") + 1, "edad2")

In [ ]:
# Reordenando y guadando en df2
df2 = df2[lista_columnas]

**Emulando relocate() de R en Python**

In [ ]:
def relocate(df, columna, after):
  lista_columnas = df.columns.to_list()
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = "edad2",
    after = "Escribe tu edad exacta"
)

### **Eliminar la fila completa**

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

## **Estandarización de variables**

### **Normalización**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# Creando df3
df3 = df2.copy()

In [ ]:
df3["edadZ"] = normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# Reubicando edadZ
df3 = relocate(
    df = df3,
    columna = "edadZ",
    after = "edad2"
)

### **Rango**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
rango.fit_transform(df3[["edad2"]])

In [ ]:
df3["edadRango"] = rango.fit_transform(df3[["edad2"]])

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadRango",
    after = "edadZ"
)

In [ ]:
df3.head()

## **Agrupaciones**

### **Numéricas**

In [ ]:
cortes = [-float("inf"), 18, 21, float("inf")]
etiquetas = ["18 o menos", "19 a 21", "Más de 21"]

In [ ]:
# Creando columna edadGR
df3["edadGR"] = pd.cut(
    df3["edad2"],
    bins = cortes,
    labels = etiquetas
)

In [ ]:
# comparando
pd.DataFrame({
    "original": df3["edad2"],
    "edadGR": df3["edadGR"]
})

### **Categóricas**

In [ ]:
# índices del df3
df3.info()

In [ ]:
# identificando los niveles
pd.unique(df3.iloc[:,7])

In [ ]:
df3.iloc[:,7].value_counts()

In [ ]:
# prueba lógica top 2 box
df3.iloc[:,7].isin(["Totalmente verdadero", "Un poco verdadero"]).value_counts()

**Dummitización**

In [ ]:
# función condicional normal
def top2box(x):
  if x in ["Totalmente verdadero", "Un poco verdadero"]:
    return 1
  else:
    return 0

In [ ]:
# aplicando la función
df3.iloc[:,7].apply(top2box)

In [ ]:
# frecuencias
df3.iloc[:,7].apply(top2box).value_counts()

**Funciones lambda**

In [ ]:
df3.iloc[:,7].apply(lambda x: 1 if x in ["Totalmente verdadero", "Un poco verdadero"] else 0).value_counts()

**Escala ordinal**

In [ ]:
import numpy as np

In [ ]:
# niveles
pd.unique(df3.iloc[:,7])

In [ ]:
condiciones = [
    df3.iloc[:,7] == "Totalmente falso",
    df3.iloc[:,7] == "Un poco falso",
    df3.iloc[:,7] == "No lo sé",
    df3.iloc[:,7] == "Un poco verdadero",
    df3.iloc[:,7] == "Totalmente verdadero"
]

opciones = [1, 2, 3, 4, 5]

In [ ]:
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [ ]:
# como se trata de un objeto de numpy, el método es distinto
np.unique(ordinal, return_counts = True)

In [ ]:
# si quisieras usar value_counts()...
pd.Series(ordinal).value_counts()

**Bucles**

Manera convencional

In [ ]:
# creando df4
df4 = df3.copy()

In [ ]:
df4.columns

In [ ]:
# crear una lista vacía
frases = []

# creando bucle para llenar vector vacío
for col in df4.columns:
  if col.startswith("Según tu"):
    frases.append(col)

In [ ]:
frases

In [ ]:
# creando otro bucle, esta vez para dummitizar
for var in frases:
  df4[var] = df4[var].apply(top2box)

Manera pythonica

In [ ]:
# creando df5
df5 = df3.copy()

In [ ]:
# crear una lista vacía
frases = []

# usando list comprehension
frases = [col for col in df5.columns if col.startswith("Según tu")]

In [ ]:
# creando el bucle para dummitizar con función lambda
for var in frases:
  df5[var] = df5[var].apply(lambda x: 1 if x in ["Totalmente verdadero", "Un poco verdadero"] else 0)

# **Tema 3: Manipulación de data frames**

## **Selección de columnas**

In [ ]:
df5["Sexo"]

In [ ]:
df5[["Sexo"]]

In [ ]:
df5[["Sexo", "edad2"]]

In [ ]:
df5.drop(columns = ["Marca temporal"], axis = 1)

In [ ]:
df5[[col for col in df5.columns if col.startswith("edad")]]

In [ ]:
df5[[col for col in df5.columns if col.endswith("00")]]

In [ ]:
df5.drop(df5.iloc[:,0:5], axis = 1)

In [ ]:
df5.filter(like = "edad", axis = 1)

## **Selección de filas**

In [ ]:
df5[df5["Sexo"] == "Mujer"]

In [ ]:
df5[df5["Sexo"] != "Hombre"]

In [ ]:
df5[df5["edad2"] >= 20]

In [ ]:
df5[
    (df5["edad2"] >= 18) &
    (df5["edad2"] <= 21) &
    (df5["Sexo"] == "Hombre")
][["Sexo", "edad2"]]

In [ ]:
df3[df3.iloc[:,7].str.contains("falso", regex = False)]

## **Cambio de nombre de columnas**

In [ ]:
df6 = df5.copy()

### **Apps**

In [ ]:
df6.info()

**Paso 1:** Crear una lista con los nuevos nombres

In [ ]:
apps = ["TikTok", "Instagram", "Facebook", "YouTube"]

In [ ]:
apps

**Paso 2:** Crear un dict que contenga los nombres antiguos como claves (key) y los nuevos como valor (value)

In [ ]:
# zip() crea tuplas (nombre antiguo, nombre nuevo)
# dict() almacena en un diccionario
apps_dict = dict(
    zip(
        [col for col in df6.columns if col.endswith("00")],
        apps
    )
)

In [ ]:
apps_dict

**Paso 3:** Reemplazar

In [ ]:
df6.rename(apps_dict, axis = 1, inplace = True)

In [ ]:
df6.info()

### **Frases**

**Paso 0:** Convertir la lista a pd.Series

In [ ]:
type(frases)

In [ ]:
frases2 = pd.Series(frases)

In [ ]:
type(frases2)

**Paso 1:** Crear una lista con los nuevos nombres

In [ ]:
# Solución de Samuel Vizcarra
# con corchetes: \[.*?\]
# sin corchetes: \[(.*?)\]
frases2 = frases2.str.extract("\\[(.+)\\]")[0]

**Paso 2:** Crear un dict que contenga los nombres antiguos como claves (key) y los nuevos como valor (value)

In [ ]:
frases_dict = dict(zip(frases, frases2))

In [ ]:
frases_dict

**Paso 3:** Reemplazar

In [ ]:
df6.rename(frases_dict, axis = 1, inplace = True)

In [ ]:
df6.info()

## **Pivotado de base de datos**

### **Pivot Longer**

In [ ]:
df7 = df6.melt(
    # identifiers
    id_vars = ["Marca temporal", "Sexo", "edad2"],
    value_vars = apps,
    var_name = "app",
    value_name = "time"
)

In [ ]:
df7.shape

In [ ]:
df7

### **Pivot Wider**

In [ ]:
df8 = df7.pivot(
    # identifiers
    index = ["Marca temporal", "Sexo", "edad2"],
    columns = "app",
    values = "time"
)

In [ ]:
# Notar que las columnas se ven raras
df8.head()

In [ ]:
# Los identifiers NO son columnas
df8.info()

In [ ]:
# Reiniciando index
df8.reset_index(inplace = True)

In [ ]:
df8.head()

In [ ]:
# Quitando nombre al grupo de columnas
df8.columns.name = None

In [ ]:
df8.head()

# **Tema 4: Detección de outliers**

**Transformación de horas a númerico**

In [ ]:
# split separa textos
"02:30:45".split(":")

In [ ]:
# función conversora
def tiempo_a_num(x):
  return int(x.split(":")[0]) + int(x.split(":")[1])/60 + int(x.split(":")[2])/60**2

In [ ]:
# probando la función
tiempo_a_num("00:30:00")

In [ ]:
df7["time"] = df7["time"].apply(tiempo_a_num)

## **Gráfica: Boxplot**

In [ ]:
import plotly.express as px

In [ ]:
px.box(df7, y = "time")

**Apertura por app**

**Usando plotly**

In [ ]:
px.box(
    # esta parte es la equivalente a ggplot()
    df7,
    x = "app",
    y = "time",
    color = "app",
    # points = "all"
).update_layout(
    # update_layout es el equivalente a theme() en ggplot
    title = "Apertura por app",
    showlegend = False,
    xaxis_title = "App",
    yaxis_title = "Tiempo (horas)",
    plot_bgcolor = "white"
).update_traces({
    "marker": {"size": 8},
    "line": {"width": 4},
    "fillcolor": "white"
})

**Usando ggplot**

In [ ]:
from plotnine import *

In [ ]:
# Los paréntesis especifican que todo forma parte de un solo statement
(
    ggplot(df7, aes(x = "app", y = "time", color = "app"))
    + geom_boxplot()
    + geom_jitter(width = 0.2, alpha = 0.5)
    + theme_minimal()
    + theme(legend_position = "none")
    + labs(
        title = "Apertura por app",
        x = "App",
        y = "Tiempo (horas)"
    )
).draw()

## **Tratamiento**

In [ ]:
df9 = df7.copy()

In [ ]:
condiciones = (
    ((df9["app"] == "TikTok")    & (df9["time"] > 16.37)) |
    ((df9["app"] == "Instagram") & (df9["time"] > 12))    |
    ((df9["app"] == "Facebook")  & (df9["time"] > 10))    |
    ((df9["app"] == "YouTube")   & (df9["time"] > 9))
)

In [ ]:
df9["outlier"] = np.where(condiciones, "outlier", "No outlier")

In [ ]:
df9["promedio"] = df9.groupby("app")["time"].transform("mean")

In [ ]:
df9["time2"] = np.where(df9["outlier"] == "outlier", df9["promedio"], df9["time"])

In [ ]:
df9

# **Tema 5: Exploratory Data Analysis (EDA)**

## **Variables categóricas**

In [ ]:
# conteo
df6["edadGR"].value_counts()

In [ ]:
# proporciones
df6["edadGR"].value_counts(normalize = True)

In [ ]:
# orden ascendente
df6["edadGR"].value_counts().sort_values()

In [ ]:
# orden descendente
df6["edadGR"].value_counts().sort_values(ascending = False)

In [ ]:
# de Series a DataFrame
df_edadGR = df6["edadGR"].value_counts().reset_index()
df_edadGR.columns = ["edadGR", "n"]

In [ ]:
# orden ascendente por columna
df_edadGR.sort_values(by = "n", ascending = True)

In [ ]:
# crear nuevas columnas calculadas
df_edadGR["prop"] = df_edadGR["n"] / df_edadGR["n"].sum()

In [ ]:
df_edadGR

## **Variables numéricas**

In [ ]:
df9["time"].describe()